# 03. 상호작용 지도 만들기

## 라이브러리 불러오기

In [1]:
import pandas as pd
import geopandas as gpd
import math

In [2]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

## 간단한 상호작용 지도 만들기

In [3]:
m_1 = folium.Map(location=[42.32, -71.0589], tiles='openstreetmap', zoom_start=10)

m_1

## 범죄 데이터 불러오기

In [4]:
# load the data
crimes = pd.read_csv('C:/Users/GBOOK/PythonGeoSpatialAnalysis/archive/crimes-in-boston/crimes-in-boston/crime.csv', encoding='latin-1')

# Drop rows with missing location
crimes.dropna(subset=['Lat', 'Long', 'DISTRICT'], inplace=True)

# Focus on Major crimes in 2018
crimes = crimes[crimes.OFFENSE_CODE_GROUP.isin([
    'Larceny', 'Auto Theft', 'Robbery', 'Larceny From Motor Vehicle', 'Residential Burglary',
    'Simple Assault', 'Harassment', 'Ballistics', 'Aggravated Assault', 'Other Burglary', 
    'Arson', 'Commercial Burglary', 'HOME INVASION', 'Homicide', 'Criminal Harassment', 
    'Manslaughter'])]
crimes = crimes[crimes.YEAR >= 2018]

# Print First Five rows of the table
crimes.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
6,I182070933,724,Auto Theft,AUTO THEFT,B2,330,NaN,2018-09-03 21:25:00,2018,9,Monday,21,Part One,NORMANDY ST,42.306072,-71.082733,"(42.30607218, -71.08273260)"
8,I182070931,301,Robbery,ROBBERY - STREET,C6,177,NaN,2018-09-03 20:48:00,2018,9,Monday,20,Part One,MASSACHUSETTS AVE,42.331521,-71.070853,"(42.33152148, -71.07085307)"
19,I182070915,614,Larceny From Motor Vehicle,LARCENY THEFT FROM MV - NON-ACCESSORY,B2,181,NaN,2018-09-02 18:00:00,2018,9,Sunday,18,Part One,SHIRLEY ST,42.325695,-71.068168,"(42.32569490, -71.06816778)"
24,I182070908,522,Residential Burglary,BURGLARY - RESIDENTIAL - NO FORCE,B2,911,NaN,2018-09-03 18:38:00,2018,9,Monday,18,Part One,ANNUNCIATION RD,42.335062,-71.093168,"(42.33506218, -71.09316781)"


In [5]:
crimes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15682 entries, 0 to 318868
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   INCIDENT_NUMBER      15682 non-null  object 
 1   OFFENSE_CODE         15682 non-null  int64  
 2   OFFENSE_CODE_GROUP   15682 non-null  object 
 3   OFFENSE_DESCRIPTION  15682 non-null  object 
 4   DISTRICT             15682 non-null  object 
 5   REPORTING_AREA       15682 non-null  object 
 6   SHOOTING             110 non-null    object 
 7   OCCURRED_ON_DATE     15682 non-null  object 
 8   YEAR                 15682 non-null  int64  
 9   MONTH                15682 non-null  int64  
 10  DAY_OF_WEEK          15682 non-null  object 
 11  HOUR                 15682 non-null  int64  
 12  UCR_PART             15666 non-null  object 
 13  STREET               15647 non-null  object 
 14  Lat                  15682 non-null  float64
 15  Long                 15682 non-null  flo

## 포인트 데이터 그리기
지도에 표현되는 데이터의 양을 줄이기 위해 주간시간대 9-18시 사이에 일어나는 강도범죄들만 표현

In [6]:
# daytime_robberies = crimes[((crimes.OFFENSE_CODE_GROUP == 'Robbery') & (crimes.HOUR.isin(range(9, 18))))]
daytime_robberies = crimes[((crimes.OFFENSE_CODE_GROUP == 'Robbery') & \
                            (crimes.HOUR.isin(range(9,18))))]

In [7]:
daytime_robberies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 231 entries, 299 to 181223
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   INCIDENT_NUMBER      231 non-null    object 
 1   OFFENSE_CODE         231 non-null    int64  
 2   OFFENSE_CODE_GROUP   231 non-null    object 
 3   OFFENSE_DESCRIPTION  231 non-null    object 
 4   DISTRICT             231 non-null    object 
 5   REPORTING_AREA       231 non-null    object 
 6   SHOOTING             1 non-null      object 
 7   OCCURRED_ON_DATE     231 non-null    object 
 8   YEAR                 231 non-null    int64  
 9   MONTH                231 non-null    int64  
 10  DAY_OF_WEEK          231 non-null    object 
 11  HOUR                 231 non-null    int64  
 12  UCR_PART             231 non-null    object 
 13  STREET               231 non-null    object 
 14  Lat                  231 non-null    float64
 15  Long                 231 non-null    flo

## folium Marker 표현하기

In [8]:
# Create a map
m_2 = folium.Map(location=[42.32, -71.0589], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
for idx, row in daytime_robberies.iterrows():
    Marker([row['Lat'], row['Long']]).add_to(m_2)
    
# Display
m_2

## MarkerCluster

In [9]:
m_3 = folium.Map(location=[42.32, -71.0859], tiles='cartodbpositron', zoom_start=13)

mc = MarkerCluster()
for idx, row in daytime_robberies.iterrows():
    if not math.isnan(row['Long']) and not math.isnan(row['Lat']):
        mc.add_child(Marker([row['Lat'], row['Long']]))
m_3.add_child(mc)

m_3

## Bubble Map

In [10]:
m_4 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

def color_producer(val):
    if val <= 12:
        return 'forestgreen'
    else:
        return 'darkred'
    
for i in range(0, len(daytime_robberies)):
    Circle(
        location=[daytime_robberies.iloc[i]['Lat'], daytime_robberies.iloc[i]['Long']],
        radius=20,
        color=color_producer(daytime_robberies.iloc[i]['HOUR'])
    ).add_to(m_4)
    
m_4